In [ ]:
# Setup, Version check and Common imports

# Python ≥3.8 is required
import sys
assert sys.version_info >= (3, 5)


# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras import layers

# to make this notebook's output stable across runs
np.random.seed(42)

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

print('Python version: ', sys.version_info)
print('TF version: ', tf.__version__)
print('Keras version: ', keras.__version__)
print('GPU is', 'available' if tf.config.list_physical_devices('GPU') else 'NOT AVAILABLE')

**RESNET 50**

**1. Load Model from Keras Applications**

In [ ]:
# Load RESNET50 from keras applications
# https://keras.io/api/applications/
# https://keras.io/guides/transfer_learning/


# The ResNet-50 Model will be used to predict the category of selected images
# https://keras.io/api/applications/resnet/#resnet50-function

# Load Model with weights trained on Imagenet

from tensorflow.keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50

modelR = ResNet50(weights="imagenet")



**2. Using ResNet50 for Inference**

In [ ]:
# Load two sample images and resize them to match the input required by ResNet

K = tf.keras.backend
images = K.constant(load_sample_images()["images"])
images_resized = layers.Resizing(height=224, width=224,
                                          crop_to_aspect_ratio=True)(images)

In [ ]:
# Print the images

plt.figure(figsize=(10, 6))
for idx in (0, 1):
    plt.subplot(1, 2, idx + 1)
    plt.imshow(images_resized[idx] / 255)
    plt.axis("off")

plt.show()



In [ ]:
# Preprocess the sample images in a specific way that the model is expecting
# Call method: resnet.preprocess_input()

inputs = applications.resnet50.preprocess_input(images_resized)



In [ ]:
# Make prediction with the pretrained model

Y_proba = modelR.predict(inputs)
Y_proba.shape



In [ ]:
# Check the top predicted classes

top_K = applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)):
    print(f"Image #{image_index}")
    for class_id, name, y_proba in top_K[image_index]:
        print(f"  {class_id} - {name:12s} {y_proba:.2%}")


Are the results correct? How do you interpret the outcomes?

**3. Transfer Learning with Resnet50**

In [ ]:
# Transfer Learning using the pretrained ResNet50
# This model will be applied to the Flowers Dataset

# Data Fetching and Loading

import pathlib

!curl -O https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
!tar -xzvf flower_photos.tgz
!rm flower_photos.tgz

os.chdir('flower_photos')
path = os.getcwd()
data_dir = pathlib.Path(path)

batch_size = 32
IMG_SIZE = (224, 224)

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=batch_size,
)

class_names = train_ds.class_names
print(class_names)

train_ds = train_ds.cache().prefetch(1)
val_ds = val_ds.cache().prefetch(1)


In [ ]:
# Load ResNet50 from Keras Applications

ResNet_base = keras.applications.resnet50.ResNet50(weights="imagenet", include_top=False, input_shape=(224,224,3))

ResNet_base.trainable = False

**Quiz 1**

Explain the meaning of the following code snippets and justify why they are relevant for this transfer learning task:


1.   include_top = False
2.   ResNet_base.trainable = False


In [ ]:

# Create the complete model

# Preprocess the images to meet what is expected by ResNet
# Add GlobalAveragePooling + Classification Head

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

inputs = keras.Input(shape=(224, 224, 3))

x = keras.applications.resnet.preprocess_input(inputs)
x = ResNet_base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(5, activation='softmax')(x)


model_TL1 = tf.keras.Model(inputs, outputs)


**Quiz 2:**

1.   Detail the changes that are performed to the images by the method preprocess_input()
2.   How many weights has the feature extraction sub-model?



In [ ]:
model_TL1.summary()

In [ ]:
# Model Compilation

optimizer = tf.keras.optimizers.Adam(epsilon=0.01)

model_TL1.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
# Training (Preferably with a GPU)

history = model_TL1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
)

In [ ]:
# Visualization of Results

import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()


Analyze results. By looking at the chart, what do you think is happening?


In [ ]:
# Top layers are well-trained and now we can unfreeze the weights of the base

ResNet_base.trainable = True

# Compile again

optimizer = tf.keras.optimizers.Adam(epsilon=0.01)

model_TL1.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
model_TL1.summary()

In [ ]:
# Resume training.
# It will take a while, even with a GPU.

history = model_TL1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
)

In [ ]:
# Visualization of Results

import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()

**4. Data Augmentation**

In [ ]:
# Add Data Augmentation layers to the previous model

# Analyze how Data Augmentation impacts performance


### Code goes here




**5. Transfer Learning for CIFAR 100**

In [ ]:
# Select another model from Keras Applications (e.g., the Xception model)

# Apply it to the CIFAR100 dataset (fine grained)

# Repeat the steps from sections 3 and 4 and analyze results.